In [1]:
!pip install psycopg2

In [572]:
import pandas as pd
import numpy as np
import psycopg2 as pg

In [573]:
# DB Connection
conn = pg.connect(host='49.50.167.136',
                                port='5432',
                                dbname='synthea_1000',
                                user='walker106',
                                password='forcebewithyou')

##-- SQL query
query = """
SELECT * FROM visit_occurrence
"""

# Get a DataFrame

query_result = pd.read_sql(query, conn)



In [574]:
# 1번 문제
query_result['visit_end_date'].astype('datetime64') - query_result['visit_start_date'].astype('datetime64') + np.timedelta64(1,'D')

0       1 days
1       1 days
2       1 days
3       1 days
4       1 days
         ...  
41805   1 days
41806   1 days
41807   1 days
41808   1 days
41809   1 days
Length: 41810, dtype: timedelta64[ns]

In [575]:
query_result['total_visit'] = query_result['visit_end_date'].astype('datetime64') - query_result['visit_start_date'].astype('datetime64') + np.timedelta64(1,'D')
query_result.loc[query_result['total_visit'] ==(query_result['visit_end_date'].astype('datetime64') - query_result['visit_start_date'].astype('datetime64') + np.timedelta64(1,'D')).max(),'person_id'].count()

1

In [576]:
#2
query = """
SELECT * FROM condition_occurrence
"""

# Get a DataFrame

condition_occurrence = pd.read_sql(query, conn)

In [577]:
query = """
SELECT * FROM concept
"""

# Get a DataFrame

concept = pd.read_sql(query, conn)

In [580]:
#merge함수를 써주기위해 condition_occurrence의 column 명 수정, id 와 상병만 남기고 merge실행
concept_name= pd.merge(condition_occurrence.rename({'condition_concept_id':'concept_id'},axis=1)[['concept_id']], concept[['concept_id','concept_name']],on='concept_id')

In [581]:
#위의 결과로 나온 정보에서 중복 제거하여 concept_name 할당
concept_name= pd.DataFrame(concept_name.loc[concept_name['concept_name']!='No matching concept']['concept_name'].unique())

In [582]:
#대소문자 구분없이 검색하기위해 검색용 column 추가
concept_name['search']= concept_name[0].str.lower()
#heart를 포함한 상병 이름 조회
concept_name[concept_name['search'].str.contains('heart')][0]
#heart가 중간 값인지 확인
include_heart = concept_name[concept_name['search'].str.contains('heart')]

In [583]:
#heart 가 맨뒤에 있을시
include_heart['search'].str.len() -5 == include_heart['search'].str.find('heart')

49     False
112    False
115     True
Name: search, dtype: bool

In [584]:
#heart 가 맨 앞에 있을시
include_heart['search'].str.find('heart') == 0

49     False
112     True
115    False
Name: search, dtype: bool

In [585]:
include_heart['search'].str.find('heart')

49     19
112     0
115    10
Name: search, dtype: int64

In [586]:
#a, b, c, d, e로 시작하는 상병이름 조회
ch_serach_name= concept_name[concept_name['search'].str.startswith('a')|concept_name['search'].str.startswith('b')|concept_name['search'].str.startswith('c')|concept_name['search'].str.startswith('d')|concept_name['search'].str.startswith('e')]

In [587]:
#a,b,c,d,e로 시작하고, heart가 포함되고, 중간에 위치한 상병 이름 조회
include_heart = ch_serach_name[ch_serach_name['search'].str.contains('heart')]
include_heart[~(include_heart['search'].str.find('heart') == 0)|(include_heart['search'].str.len() -5 == include_heart['search'].str.find('heart'))][[0]]

,0
49,Chronic congestive heart failure


In [588]:
#3
query = """
SELECT * FROM drug_exposure
"""

# Get a DataFrame

drug_exposure = pd.read_sql(query, conn)

In [589]:
#환자번호 1891866 조회
durg_table = drug_exposure.loc[drug_exposure['person_id']==1891866][['person_id','drug_concept_id','drug_exposure_start_date','drug_exposure_end_date']]

In [590]:
#복용 길이 할당
durg_table['length']= durg_table['drug_exposure_end_date'].astype('datetime64') - durg_table['drug_exposure_start_date'].astype('datetime64')

In [591]:
#datedeltime type int형으로 수정
durg_table['length'] = durg_table['length'].apply(lambda x: x.days)

In [592]:
#drug_exposure_start_date 값
min_durg = durg_table.groupby('drug_concept_id').min()

In [593]:
#drug_exposure_end_date 값
max_drug = durg_table.groupby('drug_concept_id').max()

In [594]:
#총 복용길이
durg_table_answer = durg_table.groupby('drug_concept_id').sum()

In [595]:
durg_table_answer['person_id'] = 1891866

In [596]:
# 병합
durg_table_answer = pd.concat([durg_table_answer,max_drug['drug_exposure_end_date'],min_durg['drug_exposure_start_date']],axis=1).sort_values('length',ascending=False)

In [597]:
#답에 맞게 수정
durg_table_answer.reset_index()[['person_id', 'drug_concept_id','drug_exposure_start_date','drug_exposure_end_date']]

,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_end_date
0,1891866,19009384,1959-12-01,1998-10-06
1,1891866,1539463,1990-03-13,1998-03-11
2,1891866,19030765,1988-10-18,1998-10-05
3,1891866,40213154,1989-09-12,1998-07-07
4,1891866,40213227,1993-01-05,1993-01-05


In [598]:
#4
query = """
select * from drug_exposure
where drug_concept_id in (
40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,
19126352,1539411,1332419,40163924,19030765,19106768,19075601)
"""
# Get a DataFrame

durg_table = pd.read_sql(query, conn)

In [599]:
#처방건수 table
prescription_count = durg_table.groupby('drug_concept_id').count()[['drug_exposure_id']]

In [600]:
query = """
select * from concept
where concept_id in (
40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,
19126352,1539411,1332419,40163924,19030765,19106768,19075601)
"""
# Get a DataFrame

durg_name = pd.read_sql(query, conn)

In [601]:
#durg_name table
durg_name = durg_name[['concept_id', 'concept_name']]

In [602]:
query = """SELECT * FROM drug_pair
"""
durg_pair = pd.read_sql(query, conn)

In [603]:
# durg_pair table 쿼리
durg_pair

,drug_concept_id1,drug_concept_id2
0,40213154,19078106
1,19078106,40213154
2,19009384,19030765
3,40224172,40213154
4,19127663,19009384
5,1511248,40169216
6,40169216,1511248
7,1539463,19030765
8,19126352,1539411
9,1539411,19126352


In [604]:
# pair로 작성된 약중, 두번째 약이 처방이 많이 된 첫번째 약의 약품명, 처방건수 출력
# 첫번째 약의 처방건수 정의
durg_pair['cnt_id1']= durg_pair['drug_concept_id1'].apply(lambda x : prescription_count.T[x].values[0])

In [605]:
#두번째 약의 처방건수 정의
durg_pair['cnt_id2']= durg_pair['drug_concept_id2'].apply(lambda x : prescription_count.T[x].values[0])

In [606]:
#첫번째약 이름
durg_pair['name']= durg_pair['drug_concept_id1'].apply(lambda x : durg_name.set_index('concept_id').T[x].values[0])

In [607]:
# 두번째 약의 처방이 많이 된 경우 T, 아닌경우 N
durg_pair['check']= np.where(durg_pair['cnt_id1']<durg_pair['cnt_id2'],'T','N')

In [608]:
# pair로 작성된 약중, 두번째 약이 처방이 많이 된 첫번째 약의 약품명을 처방건수 순으로 출력
durg_pair.loc[durg_pair['check']=='T'].sort_values('cnt_id1')[['name']]

,name
14,clopidogrel 75 MG Oral Tablet
13,hydrochlorothiazide 12.5 MG Oral Tablet
12,1 ML epoetin alfa 4000 UNT/ML Injection [Epogen]
11,24 HR metformin hydrochloride 500 MG Extended ...
10,amlodipine 5 MG Oral Tablet
9,simvastatin 20 MG Oral Tablet
6,120 ACTUAT fluticasone propionate 0.044 MG/ACT...
4,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...
3,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...
1,hydrochlorothiazide 25 MG Oral Tablet


In [609]:
#5 제 2형 당뇨병을 진단받은 환자

In [610]:
query=  """
SELECT * FROM condition_occurrence
WHERE condition_concept_id IN (3191208,36684827,3194332,3193274,43531010,4130162,45766052,
45757474,4099651,4129519,4063043,4230254,4193704,4304377,201826,3194082,3192767)
"""

# Get a DataFrame

diabetic = pd.read_sql(query, conn)

In [612]:
# 18세 이상의 환자(person table)
query=  """
select * from de.person
"""

# Get a DataFrame

person = pd.read_sql(query, conn)

In [614]:
# 18세 이상의 환자(person table)
diabetic_table = pd.merge(diabetic,person[['person_id', 'year_of_birth']], on= 'person_id')

In [615]:
diabetic_person = diabetic_table.loc[diabetic_table['year_of_birth']<2005][['person_id']]

In [616]:
# 확인인자 T
diabetic_person['chk'] = 'T'

In [617]:
#진단을 받은 이후 Metformin을 90일 이상 복용한 환자수

In [618]:
metformin_table = drug_exposure.loc[drug_exposure['drug_concept_id'] == 40163924]

In [619]:
# 복용기간정의
metformin_table['length']= metformin_table['drug_exposure_end_date'].astype('datetime64') - metformin_table['drug_exposure_start_date'].astype('datetime64')

<ipython-input-619-02c569b73e59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metformin_table['length']= metformin_table['drug_exposure_end_date'].astype('datetime64') - metformin_table['drug_exposure_start_date'].astype('datetime64')


In [620]:
# 복용기간 int형으로 변형
metformin_table['length'] = metformin_table['length'].apply(lambda x: x.days)

<ipython-input-620-6f899ce10554>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metformin_table['length'] = metformin_table['length'].apply(lambda x: x.days)


In [621]:
# 사람별 복용기간 계산
metformin_length= metformin_table.groupby('person_id').sum()[['length']]

In [622]:
# 90일 이상 복용시
metformin = metformin_length.loc[ metformin_length['length']>90]

In [623]:
# 18세 이상
check_person= pd.merge(metformin.reset_index(),diabetic_person,on='person_id')

In [624]:
# 총 인원수
len(check_person.loc[check_person['chk']=='T'])

30

In [625]:
#6. 의약품 처방이 변경된 패턴 추출, 빈도의 내림차순 나열

In [626]:
#5a테이블과 durg_exposure에서 person_id로 병합
pre_patten= pd.merge(diabetic[['person_id']], drug_exposure, on= 'person_id')

In [627]:
#불필요한부분 제거
pre_patten = pre_patten[['person_id', 'drug_concept_id','drug_exposure_start_date']]

In [628]:
#약품 패턴 추출위해 문자열로 라벨링
pre_patten['patten']= np.where(pre_patten['drug_concept_id'] == 19018935,'a',np.where(pre_patten['drug_concept_id'] == 1539411,'b',np.where(pre_patten['drug_concept_id'] == 1539463,'b',np.where(pre_patten['drug_concept_id'] == 19075601,'c',np.where(pre_patten['drug_concept_id'] == 19075601,'d','NA')))))

In [629]:
#필요없는 약품 제거
pre_patten= pre_patten.loc[pre_patten['patten']!='NA']

In [630]:
#처방일 데이트타임 타입으로 변경
pre_patten['drug_exposure_start_date'] = pre_patten['drug_exposure_start_date'].astype('datetime64')

In [631]:
#같은 사람, 처방일 에따른 패턴
patten = pre_patten[['person_id','drug_exposure_start_date','patten']].groupby(['person_id','drug_exposure_start_date']).apply(sum)[['patten']]

In [632]:
#같은 처방일 a b c d 순서로 정렬
patten['patten'] = patten.patten.apply(sorted).apply(set).apply("".join)

In [633]:
# 같은 처방일 패턴구분자'()' 및 추후 split에 사용할 공백 추가
def insert_option(A):
    return'('+A+') '

In [634]:
patten['patten'] = patten.patten.apply(insert_option)

In [635]:
# 같은 처방일 기준 패턴처리
result = []
for i in patten.reset_index().groupby('person_id').apply(sum)['patten'].str.split(' ').values:
    temp_result = ''
    for j in i[1:]:
        if j == init:
            continue
        else:
            temp_result= temp_result+init+'->'
            init = j
    result.append(temp_result)



In [636]:
# 결과 데이터 프레임 생성
patten_result = patten.reset_index().groupby('person_id').count()

In [637]:
patten_result['result']= result

In [638]:
#맨앞, 맨뒤 필요없는 구분자 제거
patten_result['result']= patten_result['result'].str.slice(2,-2).values

In [639]:
# 패턴별 갯수 내림차순 정렬
patten_result.groupby('result').count().iloc[:,[1]].sort_values('patten', ascending=False)

,patten
result,
(b),7
(bc),3
(a),1
(a)->(ab),1
(a)->(b),1
(b)->(bc),1
(b)->(bc)->(b)->(bc)->(b)->(bc)->(b)->(bc),1
(b)->(c),1
(b)->(c)->(bc)->(a)->(abc)->(c)->(abc)->(c)->(abc),1


In [640]:
#7
#파싱 및 전처리
query=  """
select * from de.clinical_note
"""

# Get a DataFrame

note = pd.read_sql(query, conn)

In [641]:
# \n 구분자로 자르기
len(note.note[0].split('\n'))

64

In [642]:
k = note.note[14].split('\n')

In [643]:
# ---, ===, CONTINUING의 위치 할당
init_pos = [i for i, x in enumerate(k) if x == '===============']
pos = [i for i, x in enumerate(k) if x == '--------------------------------------------------------------------------------']
con_pos = [i for i, x in enumerate(k) if x == 'CONTINUING']
print(init_pos)
print(pos)
print(con_pos)
        

[1]
[8, 11, 35, 45]
[36]


In [644]:
# 위치기준으로 초기 데이터에서 파싱
pd.DataFrame(k[init_pos[0]+1:pos[0]])[0].str.split(':',expand=True)

,0,1
0,Race,White
1,Ethnicity,Non-Hispanic
2,Gender,M
3,Age,55
4,Birth Date,1965-04-22
5,Marital Status,M


In [645]:
#알러지 데이터 안쓸듯 함
k[pos[0]:pos[1]]

['--------------------------------------------------------------------------------',
 'ALLERGIES:',
 'No Known Allergies']

In [646]:
# 규칙성 찾아낼것
k[pos[1]+1:pos[2]]

['ENCOUNTER',
 '1984-06-07 : Encounter at FALMOUTH HOSPITAL : Encounter for Major depression, single episode',
 'Type: ambulatory',
 '   ',
 '   MEDICATIONS:',
 '   ',
 '   CONDITIONS:',
 '   ',
 '   CARE PLANS:',
 '  1984-06-07 : Mental health care plan',
 '                         Reason: Major depression, single episode',
 '                         Activity: Initial psychiatric interview with mental status and evaluation',
 '   ',
 '   REPORTS:',
 '   ',
 '   OBSERVATIONS:',
 '   ',
 '   PROCEDURES:',
 '   ',
 '   IMMUNIZATIONS:',
 '   ',
 '   IMAGING STUDIES:',
 '   ']

In [647]:
#CONTINUING 위아래 ---- 사이 제거 
res = []
for i in pos:
    if con_pos[0] < i:
        continue
    else:
        k[init_pos[0]+1:pos[0]]
        res.append(i)

In [648]:
res

[8, 11, 35]

In [649]:

# 랜덤한수 person_id 에 부여하기 (중복제거)

# 내원번호 부여



In [650]:
# 데이터 생성